In [1]:
import transformers

model_id="AXCXEPT/EZO-Common-9B-gemma-2-it"
tokenizer=transformers.AutoTokenizer.from_pretrained(model_id)

/home/setup/miniconda3/envs/sft/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
#os.environ["CUDA_VISIBLE_DEVICES"]=""
model=transformers.AutoModelForCausalLM.from_pretrained(model_id)

Loading checkpoint shards: 100%|██████████| 4/4 [00:02<00:00,  1.81it/s]


In [3]:
message=[ { "role": "user", "content": "それでさ、あの迷亭って男ってめっちゃ酔狂だよね。役にも立たない嘘ばっかり並べてさ。あんな変な人に初めて会ったよ。" }, { "role": "assistant", "content": "ほんとだよね。なんかあの人の話って全然信じられないし、ちょっと変わってるよね。でも、なんか面白いからつい聞いちゃうんだよな。" }, { "role": "user", "content": "そうそう、なんか独特の魅力があるんだよね。まあ、嘘ばっかりじゃ困るけど、ちょっとした笑いを提供する存在って感じかな。" }, { "role": "assistant", "content": "うん、あの人のトークって笑えるし、なんか癒されるんだよね。嘘でも笑えるなら別にいいやって思うこともあるよ。" }, { "role": "user", "content": "なんか、そうやって笑わせてくれる存在って大事だよね。他にそんな人あんまりいないし。" }, { "role": "assistant", "content": "そうだね。みんな真面目な話ばっかりだし、ちょっと笑わせてくれる人って貴重だよ。" }, { "role": "user", "content": "ほんとだよね。そういう人がいると、なんか気持ちが軽くなるっていうか。" }, { "role": "assistant", "content": "そうそう、迷亭みたいな人がいるだけで、気分が明るくなるんだよね。" } ]
prompt=tokenizer.apply_chat_template(message,tokenize=False)

train_text_list=[
prompt

]
train_text_list

['<bos><start_of_turn>user\nそれでさ、あの迷亭って男ってめっちゃ酔狂だよね。役にも立たない嘘ばっかり並べてさ。あんな変な人に初めて会ったよ。<end_of_turn>\n<start_of_turn>model\nほんとだよね。なんかあの人の話って全然信じられないし、ちょっと変わってるよね。でも、なんか面白いからつい聞いちゃうんだよな。<end_of_turn>\n<start_of_turn>user\nそうそう、なんか独特の魅力があるんだよね。まあ、嘘ばっかりじゃ困るけど、ちょっとした笑いを提供する存在って感じかな。<end_of_turn>\n<start_of_turn>model\nうん、あの人のトークって笑えるし、なんか癒されるんだよね。嘘でも笑えるなら別にいいやって思うこともあるよ。<end_of_turn>\n<start_of_turn>user\nなんか、そうやって笑わせてくれる存在って大事だよね。他にそんな人あんまりいないし。<end_of_turn>\n<start_of_turn>model\nそうだね。みんな真面目な話ばっかりだし、ちょっと笑わせてくれる人って貴重だよ。<end_of_turn>\n<start_of_turn>user\nほんとだよね。そういう人がいると、なんか気持ちが軽くなるっていうか。<end_of_turn>\n<start_of_turn>model\nそうそう、迷亭みたいな人がいるだけで、気分が明るくなるんだよね。<end_of_turn>\n']

In [4]:
import datasets
ds=datasets.Dataset.from_list([{"text":t} for t in train_text_list])
from trl import DataCollatorForCompletionOnlyLM

# response_templateは必須指定
response_template = "<end_of_turn>\n<start_of_turn>model\n"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)
def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['text'])):
        text = f"{example['text'][i]}"
        output_texts.append(text)
    return output_texts

In [5]:
from transformers import TrainingArguments
from trl import SFTTrainer

# SFTTrainerはTrainingArgumentsを使用することができる。
# 指定しない場合、TrainingArgumentsのデフォルトが指定される。
args = TrainingArguments(
    output_dir='./output',
    num_train_epochs=1,
    gradient_accumulation_steps=8,
    per_device_train_batch_size=8,
    save_strategy="no",
    logging_steps=1,
    lr_scheduler_type="constant",
    save_total_limit=1,
    fp16=True,
)


# data_collatorが指定されていない場合、以下のようにDataCollatorForLanguageModelingがmlm=Falseで使われる。
# つまり通常のCausal LMを学習することになる。
# if data_collator is None:
#     data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
# packing=False（default）ではdataset_text_fieldかformatting_funcを指定する必要あり
trainer = SFTTrainer(
    model,
    args=args,
    train_dataset=ds,
    formatting_func=formatting_prompts_func,
    max_seq_length=1024,
    data_collator=collator,
)


[2024-09-19 19:37:00,852] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/home/setup/miniconda3/envs/sft/compiler_compat/ld: warning: libpthread.so.0, needed by /usr/local/cuda-12.3/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/setup/miniconda3/envs/sft/compiler_compat/ld: warning: libstdc++.so.6, needed by /usr/local/cuda-12.3/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/s

In [6]:

tokenizer.pad_token = tokenizer.eos_token

#loss maskのcheck
from torch.utils.data import DataLoader
print(tokenizer.decode(trainer.train_dataset[0]['input_ids']))


loader = DataLoader(trainer.train_dataset, collate_fn=collator, batch_size=8)
batch = next(iter(loader))
print(batch['labels'][0])

<bos><bos><start_of_turn>user
それでさ、あの迷亭って男ってめっちゃ酔狂だよね。役にも立たない嘘ばっかり並べてさ。あんな変な人に初めて会ったよ。<end_of_turn>
<start_of_turn>model
ほんとだよね。なんかあの人の話って全然信じられないし、ちょっと変わってるよね。でも、なんか面白いからつい聞いちゃうんだよな。<end_of_turn>
<start_of_turn>user
そうそう、なんか独特の魅力があるんだよね。まあ、嘘ばっかりじゃ困るけど、ちょっとした笑いを提供する存在って感じかな。<end_of_turn>
<start_of_turn>model
うん、あの人のトークって笑えるし、なんか癒されるんだよね。嘘でも笑えるなら別にいいやって思うこともあるよ。<end_of_turn>
<start_of_turn>user
なんか、そうやって笑わせてくれる存在って大事だよね。他にそんな人あんまりいないし。<end_of_turn>
<start_of_turn>model
そうだね。みんな真面目な話ばっかりだし、ちょっと笑わせてくれる人って貴重だよ。<end_of_turn>
<start_of_turn>user
ほんとだよね。そういう人がいると、なんか気持ちが軽くなるっていうか。<end_of_turn>
<start_of_turn>model
そうそう、迷亭みたいな人がいるだけで、気分が明るくなるんだよね。<end_of_turn>

tensor([  -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
          -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
          -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
          -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
          -100,   